In [0]:
%pip install docling
%pip install -qU langchain-text-splitters
dbutils.library.restartPython()

  Using cached pydantic-2.11.6-py3-none-any.whl.metadata (67 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/15.1 MB ? eta -:--:--
   ━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/15.1 MB 28.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.9 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/4.9 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/12.4 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 170.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.0 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━

In [0]:
from docling.document_converter import DocumentConverter
from langchain_text_splitters import MarkdownHeaderTextSplitter
from pyspark.sql.functions import monotonically_increasing_id

In [0]:
# List of PDFs to convert to Markdown
bronze_volume = "/Volumes/users/david_hurley/vehicle_warranty/" # WHERE PDFS LIVE
file_paths = [file[0].replace("dbfs:", "") for file in dbutils.fs.ls(bronze_volume)]

In [0]:
# For each PDF use Docling to convert to Markdown for each page and use Langchain to split on headers to smaller chunks
converter = DocumentConverter()
markdown_data_for_all_docs = []

for file in file_paths:
  result = converter.convert(file)

  markdown_document = result.document.export_to_markdown()

  headers_to_split_on = [
      ("#", "Header 1"),
      ("##", "Header 2"),
      ("###", "Header 3"),
  ]

  markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
  md_header_splits = markdown_splitter.split_text(markdown_document)

  # Add more columns like Page Number or URL as metadata for AI to filter on
  markdown_data_for_all_docs.extend([{"chunked_markdown": x.page_content} for x in md_header_splits])

df = spark.createDataFrame(markdown_data_for_all_docs)

In [0]:
# Add unique ID and save as table
df = df.withColumn("id", monotonically_increasing_id())
df.write.option("mergeSchema", "true").mode("overwrite").saveAsTable("users.david_hurley.markdown")